In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [3]:
# df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/LFM-1b_LEs.txt',sep='\t',names=['user_id', 'artist_id','album_id','track_id','timestamp'],nrows=300000000)

In [4]:
# df.drop(columns=['album_id','artist_id','timestamp'],inplace=True)
# user_track_counts = df.groupby(['user_id', 'track_id']).size().reset_index(name='listen_count')
# df = user_track_counts.drop_duplicates(subset=['user_id', 'track_id'])
# df.head()

In [5]:
# num_users = (df['user_id'].nunique())
# num_items = (df['track_id'].nunique())
# print(num_users)
# print(num_items)

In [6]:
# print(df.shape)
# filtered_df = df.groupby('user_id').filter(lambda x: x['listen_count'].count() >= 5)
# filtered_df = filtered_df.groupby('track_id').filter(lambda x: x['listen_count'].count() >= 5)

# print("Filtered DataFrame:")
# print(filtered_df.shape)

In [7]:
# filtered_df.to_csv(r'/content/drive/MyDrive/집교 2_Team P/실험/using_100m/filtered_df.csv', index=False)

In [8]:
# # listen_count 기준으로 나누어서 시각화
# bins = [0,1,2,3,4,5,6,7,8,9,10000]#20,30,40,50,60,70,80,90,100,1000]
# labels = [1,2,3,4,5,6,7,8,9,10]#'20','30','40','50','60','70','80','90','100','1000']

# filtered_df['listen_count_bin'] = pd.cut(filtered_df['listen_count'], bins=bins, labels=labels)

# # seaborn을 사용하여 시각화
# sns.countplot(x='listen_count_bin', data=filtered_df)
# plt.title('Listen Count Distribution')
# plt.xlabel('Listen Count Range')
# plt.ylabel('Count')
# plt.show()

In [9]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/실험/using_100m/filtered_df.csv')

In [10]:
# prompt: Df에서 nan 값 있는 row 제거하기

df.dropna(inplace=True)


In [11]:
# df.head()

In [12]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])

In [13]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)

    return users, items, ratings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings = df_to_tensor(train_df)
test_users, test_items, test_ratings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings)
test_data = TensorDataset(test_users, test_items, test_ratings)

train_loader = DataLoader(train_data, batch_size=2048, shuffle=True)
test_loader = DataLoader(test_data, batch_size=2048, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

24941
2342715


In [14]:
# NCF 모델 정의
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 2, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        x = torch.cat((user_embedding, item_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      cnt=0
      for user, item, rating in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          # cnt+=1
          optimizer.zero_grad()
          user, item, rating = user.to(device), item.to(device), rating.to(device)  # GPU로 이동
          output = model(user, item)
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()
        # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
          # if cnt%10000==0:
          #   model.eval()
          #   all_predictions = []
          #   with torch.no_grad():
          #       for user, item, _ in test_loader:
          #           user, item = user.to(device), item.to(device)  # GPU로 이동
          #           output = model(user, item)
          #           all_predictions.append(output)

          #   # RMSE 계산
          #   predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
          #   rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
          #   print()
          #   print(f'mini-batch {cnt}, RMSE on test set: {rmse}')
          #   model.train()
          #   avg_loss = total_loss / cnt
          #   print(f'Mini-batch {cnt}/59275, Avg. Loss: {avg_loss:.4f}')
      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _ in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item = user.to(device), item.to(device)  # GPU로 이동
              output = model(user, item)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [15]:
# train_(64,10)

In [16]:
train_(512,20)

Epoch 1/20: 100%|██████████| 29638/29638 [53:41<00:00,  9.20it/s]


Epoch 1/20, Avg. Loss: 5.8047


Testing Epoch 1: 100%|██████████| 7410/7410 [03:58<00:00, 31.06it/s]


Epoch 1/20, RMSE on test set: 2.3884901647518455


Epoch 2/20: 100%|██████████| 29638/29638 [56:33<00:00,  8.73it/s]


...

Epoch 19/20, RMSE on test set: 2.686923574230527


Epoch 20/20: 100%|██████████| 29638/29638 [57:25<00:00,  8.60it/s]


Epoch 20/20, Avg. Loss: 0.9269


Testing Epoch 20: 100%|██████████| 7410/7410 [04:04<00:00, 30.26it/s]


Epoch 20/20, RMSE on test set: 2.68189226491024
